<h3>Package Installations</h3>

In [ ]:
%%shell

pip install cython
# Install pycocotools, the version by default in Colab
# has a bug fixed in https://github.com/cocodataset/cocoapi/pull/354
pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'

  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip-req-build-_k9380ip
  Running command git clone -q https://github.com/cocodataset/cocoapi.git /tmp/pip-req-build-_k9380ip
  Created wheel for pycocotools: filename=pycocotools-2.0-cp36-cp36m-linux_x86_64.whl size=266457 sha256=1dc8187d5a248eb090b7e62add8e38ad4e65f4a3178cd15be042dfbf3d6f62f1
  Stored in directory: /tmp/pip-ephem-wheel-cache-vakxftq4/wheels/90/51/41/646daf401c3bc408ff10de34ec76587a9b3ebfac8d21ca5c3a
Successfully built pycocotools
  Found existing installation: pycocotools 2.0
    Uninstalling pycocotools-2.0:
      Successfully uninstalled pycocotools-2.0


<h3>Dataloading and Dataset preparation</h3>

In [ ]:
import pandas as pd
import ast

ANNOTATION_FILEPATH = '/content/drive/My Drive/wheat_detection/data/'
TRAIN_FILEPATH = ANNOTATION_FILEPATH+'train_split.csv'
VAL_FILEPATH = ANNOTATION_FILEPATH+'val_split.csv'
train_split_df = pd.read_csv(TRAIN_FILEPATH)
val_split_df = pd.read_csv(VAL_FILEPATH)
train_split_df['bbox'] = train_split_df['bbox'].apply(ast.literal_eval)
val_split_df['bbox'] = val_split_df['bbox'].apply(ast.literal_eval)

print(train_split_df.shape)
print(val_split_df.shape)

(3005, 2)
(368, 2)


In [ ]:
import os
import torch
from PIL import Image

class WheatDataset(torch.utils.data.Dataset):
    def __init__(self, root, image_ids, bboxes, transforms=None):
        self.root = root
        self.image_ids = image_ids
        self.bboxes = bboxes
        self.transforms = transforms
    
    def __getitem__(self, index):
        img_path = os.path.join(self.root, self.image_ids[index]+'.jpg')
        img = Image.open(img_path).convert("RGB")
        boxes = self.bboxes[index]
        # there is only one class
        labels = torch.ones((len(boxes),), dtype=torch.int64)
        # suppose all instances are not crowd
        iscrowd = torch.zeros((len(boxes),), dtype=torch.int64)
        
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        
        image_id = torch.tensor([index])
        
        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd

        if self.transforms is not None:
            img, target = self.transforms(img, target)

        return img, target
    
    def __len__(self):
        return len(self.image_ids)

In [ ]:
%%shell

# Download TorchVision repo to use some files from
# references/detection
git clone https://github.com/pytorch/vision.git
cd vision
git checkout v0.6.1

cp references/detection/utils.py ../
cp references/detection/transforms.py ../
cp references/detection/coco_eval.py ../
cp references/detection/engine.py ../
cp references/detection/coco_utils.py ../

fatal: destination path 'vision' already exists and is not an empty directory.
HEAD is now at fe36f06 .circleci: Regenerate configs


In [ ]:
import transforms as T
import utils

def get_transform(train):
    transforms = []
    # converts the image, a PIL image, into a PyTorch Tensor
    transforms.append(T.ToTensor())
    if train:
        # during training, randomly flip the training images
        # and ground-truth for data augmentation
        transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)

In [ ]:
DATA_ROOT='/content/drive/My Drive/wheat_detection/data/train'

train_image_ids = train_split_df['image_id'].to_list()
train_bboxes = train_split_df['bbox'].to_list()

val_image_ids = val_split_df['image_id'].to_list()
val_bboxes = val_split_df['bbox'].to_list()

#create datasets
train_dataset = WheatDataset(DATA_ROOT, train_image_ids, train_bboxes, get_transform(train=True))
val_dataset =  WheatDataset(DATA_ROOT, val_image_ids, val_bboxes, get_transform(train=False))



# define training and validation data loaders
train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=4, shuffle=True, num_workers=4,
    collate_fn=utils.collate_fn)

val_loader = torch.utils.data.DataLoader(
    val_dataset, batch_size=4, shuffle=False, num_workers=4,
    collate_fn=utils.collate_fn)

<h3>Define model and start training</h3>

In [ ]:
import torch
import torchvision
from torchvision.models.detection.backbone_utils import resnet_fpn_backbone
from torchvision.models.detection import FasterRCNN

def fasterrcnn_101_fpn(num_classes, pretrained_backbone=True, trainable_backbone_layers=3, **kwargs):
    backbone_fpn = resnet_fpn_backbone(backbone_name='resnet101', pretrained=pretrained_backbone, trainable_layers=trainable_backbone_layers)
    model = FasterRCNN(backbone_fpn, num_classes=num_classes, **kwargs)
    return model

In [ ]:
PRETRAINED_MODEL_PATH = '/content/drive/My Drive/wheat_detection/checkpoints/faster_rcnn_resnet101/exp1/faster_rcnn_101_fpn_epoch15.pth'

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

NUM_CLASSES = 2

#get the model using helper function
model = fasterrcnn_101_fpn(NUM_CLASSES, pretrained_backbone=False)

#load model weights
model.load_state_dict(torch.load(PRETRAINED_MODEL_PATH))

# move model to the right device
model.to(device)

# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.001, momentum=0.9, weight_decay=0.0005)

# and a learning rate scheduler which decreases the learning rate by
#10x every 3 epochs
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=8, gamma=0.1)

In [ ]:
from engine import train_one_epoch, evaluate

MODEL_SAVE_PATH='/content/drive/My Drive/wheat_detection/checkpoints/faster_rcnn_resnet101/exp2/'


# let's train it for 20 epochs
num_epochs = 20

for epoch in range(num_epochs):
    # train for one epoch, printing every 10 iterations
    train_one_epoch(model, optimizer, train_loader, device, epoch, print_freq=10)
    # update the learning rate
    lr_scheduler.step()
    # evaluate on the test dataset
    evaluate(model, val_loader, device=device)
    torch.save(model.state_dict(), MODEL_SAVE_PATH+"faster_rcnn_101_fpn_latest.pth")
    if(epoch+1) % 5 == 0:
        torch.save(model.state_dict(), MODEL_SAVE_PATH+"faster_rcnn_101_fpn_epoch{}.pth".format(epoch+1))

/usr/local/lib/python3.6/dist-packages/torchvision/ops/boxes.py:101: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:766.)
  keep = keep.nonzero().squeeze(1)


Epoch: [0]  [  0/752]  eta: 0:38:40  lr: 0.000002  loss: 0.7162 (0.7162)  loss_classifier: 0.1870 (0.1870)  loss_box_reg: 0.4251 (0.4251)  loss_objectness: 0.0388 (0.0388)  loss_rpn_box_reg: 0.0652 (0.0652)  time: 3.0851  data: 1.3951  max mem: 4133
Epoch: [0]  [ 10/752]  eta: 0:11:19  lr: 0.000016  loss: 0.6527 (0.6604)  loss_classifier: 0.1870 (0.1827)  loss_box_reg: 0.3812 (0.3906)  loss_objectness: 0.0285 (0.0291)  loss_rpn_box_reg: 0.0575 (0.0582)  time: 0.9164  data: 0.1433  max mem: 4551
Epoch: [0]  [ 20/752]  eta: 0:09:55  lr: 0.000029  loss: 0.6527 (0.6678)  loss_classifier: 0.1876 (0.1868)  loss_box_reg: 0.3815 (0.3931)  loss_objectness: 0.0257 (0.0305)  loss_rpn_box_reg: 0.0552 (0.0574)  time: 0.6994  data: 0.0174  max mem: 4551
Epoch: [0]  [ 30/752]  eta: 0:09:20  lr: 0.000042  loss: 0.6798 (0.6806)  loss_classifier: 0.2026 (0.1909)  loss_box_reg: 0.4088 (0.4012)  loss_objectness: 0.0269 (0.0318)  loss_rpn_box_reg: 0.0552 (0.0567)  time: 0.7003  data: 0.0163  max mem: 4551
